In [ ]:
import matplotlib.pyplot as plt
import yfinance as yf
from pytrends.request import TrendReq
from datetime import date
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
import mplcyberpunk

$$
S_{t + 1} = 
\left\{  
  \begin{array}{ c l }
    1, & \quad \textrm{if } \frac{P_{t - 1}}{P{t}} - 1 > 50\% \\
    0                 & \quad \textrm{otherwise}
  \end{array}
\right.$$

In [ ]:
def pesquisa_x_mtum(ticker, palavra, tempo_comeco, periodos, maiores = False):
    
    hoje = date.today()
    comeco= hoje - relativedelta(months=25)
    comeco_trend = hoje - relativedelta(months=24)

    dados_acao = yf.download(f"{ticker}", start=comeco, end=hoje, progress=False)['Adj Close']
    dados_acao = dados_acao.resample('w').last()
    mtum1 = dados_acao.pct_change()
    mtum1 = mtum1.iloc[3:]
    mtum1 = mtum1.dropna()
    mtum1.name = "mtum1m"
    dados_acao = dados_acao.iloc[4:-2]

    req = TrendReq(hl='en-US', tz=360)
    req.build_payload(palavra, cat = 0, timeframe = f'{comeco_trend} {hoje}', geo = "BR", gprop = "")
    dados_procura = req.interest_over_time()
    dados_procura = dados_procura[f'{palavra[0]}']
    dados_procura = dados_procura.replace(0, 0.0001)
    
    cor = pd.merge(dados_procura,dados_acao, right_index=True, left_index=True).pct_change()
    cor = pd.merge(cor, mtum1, right_index=True, left_index=True).dropna()
    
    cor.columns = [f'{palavra[0]}' , f'{ticker}', "mtum1m"]
    cor[[f'{palavra[0]}', "mtum1m"]] = cor[[f'{palavra[0]}', "mtum1m"]].shift(periods=periodos)
    cor = cor.dropna()
    
    if maiores == True:
        
        cor = cor.sort_values(f"{palavra[0]}", ascending = False)
        cor = cor[cor[f'{palavra[0]}'] > 0.5]
        
    
    pct_cor = cor.apply(lambda x : 1 if x[f'{ticker}'] > 0 and x['mtum1m'] > 0 or 
                        x[f'{ticker}'] < 0 and x['mtum1m'] < 0 else 0, axis=1)
    
    pct_cor = pct_cor.sum()/len(pct_cor)
    retorno_min = cor[f'{ticker}'].min()
    retorno_max = cor[f'{ticker}'].max()
    retorno_med = cor[f'{ticker}'].mean()
    
    cor[f"{palavra[0]}"] = cor[f"{palavra[0]}"].map(lambda x: str(round(x * 100, 2)) + "%")
    cor[f'{ticker}'] = cor[f'{ticker}'].map(lambda x: str(round(x * 100, 2)) + "%")
    cor[f'mtum1m'] = cor[f'mtum1m'].map(lambda x: str(round(x * 100, 2)) + "%")
    
    cor.columns = [f"Pesquisa {palavra[0]}", "Rent período seguinte", "Rent semana hype"]
    
    cor = cor[[f"Pesquisa {palavra[0]}", "Rent semana hype", "Rent período seguinte"]]
    
    display(cor)
    
    #print("A quantidade de semanas com mesmo sinal da semana do hype é: ",round(pct_cor*100,2),"%")

In [ ]:
def pesquisa_x_vol(ticker, palavra, tempo_comeco, periodos, maiores = False):
    
    hoje = date.today()
    comeco= hoje - relativedelta(years=tempo_comeco)

    dados_acao = yf.download(f"{ticker}", start=comeco, end=hoje, progress=False)['Adj Close']
    dados_acao = dados_acao.resample('w').last()
    dados_acao = dados_acao.iloc[:-2]


    req = TrendReq(hl='en-US', tz=360)
    req.build_payload(palavra, cat = 0, timeframe = f'{comeco} {hoje}', geo = "BR", gprop = "")
    dados_procura = req.interest_over_time()
    dados_procura = dados_procura[f'{palavra[0]}']
    dados_procura = dados_procura.replace(0, 0.0001)
    
    cor = pd.merge(dados_procura,dados_acao, right_index=True, left_index=True).pct_change().dropna()
    cor.columns = [f'{palavra[0]}' , f'{ticker}']
    cor[f'{palavra[0]}'] = cor[f'{palavra[0]}'].shift(periods=periodos)
    cor = cor.dropna()
        
    cor = cor.sort_values(f"{palavra[0]}", ascending = False)
    retornos_palavra_no_hype = cor.head(5)
    
    cor['classe'] = ['normal'] * len(cor)
    
    cor.loc[cor.index.isin(retornos_palavra_no_hype.index), "classe"] = "retorno hype"
    
    colors = {'normal':'tab:blue', 'retorno hype':'tab:pink'}
    
    plt.style.use("cyberpunk")
    
    media_amostra = cor[f"{ticker}"].mean()
    
    desvio_padrao = cor[f"{ticker}"].std()
    
    fig, ax = plt.subplots()
    
    ax.scatter(cor[f"{palavra[0]}"], cor[f"{ticker}"], c=cor['classe'].map(colors))
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    ax.grid(False)
    plt.axhline(media_amostra, **{"color":"white"})
    plt.axhline(media_amostra + desvio_padrao, **{"color":"white", "linestyle": "--"})
    plt.axhline(media_amostra + desvio_padrao * 2, **{"color":"white", "linestyle": "--"})
    plt.axhline(media_amostra - desvio_padrao, **{"color":"white", "linestyle": "--"})
    plt.axhline(media_amostra - desvio_padrao * 2, **{"color":"white", "linestyle": "--"})
    plt.title(f"Retornos {palavra[0]} x Pesquisa {palavra[0]}")
    
    desvio_padrao = cor[f"{ticker}"].std()
    
    media_absoluta_amostra = cor[f"{ticker}"].abs().mean()
    
    media_asboluta_5 = retornos_palavra_no_hype[f"{ticker}"].abs().mean()
    
    media_absoluta_amostra = round(media_absoluta_amostra * 100, 2)
    
    media_asboluta_5 = round(media_asboluta_5 * 100, 2)
    
    print(f"A variação média absoluta da amostra é {media_absoluta_amostra}%")
    print(f"A variação média absoluta {periodos} semana(s) depois do hype é {media_asboluta_5}%")

    


# Como prever o retorno de ações na semana seguinte
### Python + Google Trends

* Pesquisas semanais do ticker

https://trends.google.com.br/trends/explore?date=2022-02-20%202024-02-20&geo=BR&q=PETR&hl=pt

# Primeira pergunta

* O que acontece na semana da pesquisa com a ação?

In [ ]:
pesquisa_x_mtum(ticker = "MGLU3.SA", 
                 palavra = ["MGLU3"], 
                 tempo_comeco = 2,
                 periodos = 0,
               #maiores = True
               )

## E na semana seguinte, o que acontece?
* A ação tende a subir ou cair quando o número de pequisa aumenta?

In [ ]:
for semanas in range(0, 4):

    pesquisa_x_mtum(ticker = "WEGE3.SA", 
                     palavra = ["WEGE3"], 
                     tempo_comeco = 2,
                     periodos = semanas,
                   maiores = True
                   )

# Quando a ação sobe em pesquisas, o que acontece com a vol?

* Talvez tentar acertar pra onde a ação vai não seja possível, porém podemos operar volatilidade!

In [ ]:
pesquisa_x_vol(ticker = "PETR4.SA", 
                 palavra = ["PETR4"], 
                 tempo_comeco = 2,
                 periodos = 1
                )